This notebook is for obtaining up-to-date protein domain information for the 1.1.3.15 sequences.<br/><br/>Copyright (C) 2020-2021  Martin Engqvist Lab<br/>This program is free software: you can redistribute it and/or modify<br/>it under the terms of the GNU General Public License as published by<br/>the Free Software Foundation, either version 3 of the License, or<br/>(at your option) any later version.<br/>This program is distributed in the hope that it will be useful,<br/>but WITHOUT ANY WARRANTY; without even the implied warranty of<br/>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br/>GNU General Public License for more details.<br/>You should have received a copy of the GNU General Public License<br/>along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


folders = [join(RAW_EXTERNAL, 'brenda_2017_1'), 
           join(INTERMEDIATE, 'brenda_2017_1'), 
           join(FINAL, 'brenda_2017_1'), 
           join(RAW_EXTERNAL, 'brenda_2019_2'), 
           join(INTERMEDIATE, 'brenda_2019_2'), 
           join(FINAL, 'brenda_2019_2')]

for f in folders:
    if not exists(f):
        os.makedirs(f)


print('Standard variables loaded, you are good to go!')

Standard variables loaded, you are good to go!


In [2]:
import pandas as pd
import re
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
import time
from orgtools import topfunctions

from Bio import SeqIO, AlignIO
from Bio.Align.Applications import MuscleCommandline
import io
from tqdm import tqdm

from unirep.run_inference import BatchInference
import tensorflow as tf

import multiprocessing

# Overview
The Pfam domains extracted in the previous notebook were only for the 2019_2 version of BRENDA, and only for sequences at the 90% identity level. To enable a detailed comparsison between the EC 1.1.3.15 enzyme family in the BRENDA versions 2017_1 and 2019_2 I go through all the identifiers and download up-to-date information directly from Pfam.

### Extract identifiers from the 2017.1 version of BRENDA
Obtain information for these such as organism name, lineage, growth pH and growth temperature.

In [3]:
def get_uid_from_fasta(filepath):
    '''
    Go through fasta file from BRENDA and extract the uid, organism and ec number.
    '''
    data = {}

    # open file and go through it
    with open(filepath, 'r') as f:
        for line in f:

            # only look at header lines
            if line.startswith('>'):
                line_data = line.lstrip('>').rstrip().split(';')
                uid = line_data[0]
                ec = line_data[2]
                org = ' '.join(line_data[3].split()[:2])

                # add ec key if not present
                if data.get(ec) is None:
                    data[ec] = {}

                if data[ec].get(org) is None:
                    data[ec][org] = []

                # add uid to data structure
                data[ec][org].append(uid)

    return data



def write_outfile(data, filepath):
    '''
    Write the parsed data to an outfile.
    '''
    with open(filepath, 'w') as f:
        f.write('uid\tec\n')

        for ec in sorted(data.keys()):
            for org in sorted(data[ec].keys()):
                for entry in data[ec][org]:
                    f.write('%s\t%s\n' % (entry, ec))



def write_info_outfile(data, filepath):
    '''
    Write the parsed data to an outfile.
    '''
    # first collect all organism names
    all_ids = []
    for ec in sorted(data.keys()):
        for org in sorted(data[ec].keys()):
            all_ids.extend(data[ec][org])


    # get taxid and lineage data
    properties_object = topfunctions.Properties(all_ids)
    properties_object.flatfile(filepath)



infile = join(INTERMEDIATE, 'brenda_2017_1', '1_1_3__BRENDA_sequences_2017_1.fasta')
outfile1 = join(FINAL, 'brenda_2017_1', 'ec_uid_org_from_fasta_2017_1.tsv')
outfile2 = join(FINAL, 'brenda_2017_1', '1-1-3-n_identifier_info_2017_1.tsv')

if (not exists(outfile1)) or (not exists(outfile2)):
    data = get_uid_from_fasta(filepath=infile)
    write_outfile(data, filepath=outfile1)
    write_info_outfile(data, filepath=outfile2)

else:
    print('Outfiles exist, skipping.')

Outfiles exist, skipping.


### Extract identifiers from the 2019.2 version of BRENDA
Obtain information for these such as organism name, lineage, growth pH and growth temperature.

In [4]:
def make_fasta():
    '''
    Go through and collect sequnces from 1.1.3.n and
    assemble into a new file.
    '''
    all_data = []
    for fi in sorted(os.listdir(join(RAW_EXTERNAL, 'brenda_2019_2', 'sequence_data'))):
        if fi.startswith('1.1.3.'):
            with open(join(RAW_EXTERNAL, 'brenda_2019_2', 'sequence_data', fi), 'r') as f:
                for line in f:
                    all_data.append(line)

    with open(join(INTERMEDIATE, 'brenda_2019_2', '1_1_3__BRENDA_sequences_2019_2.fasta'), 'w') as f:
        f.write('\n'.join(all_data))



def filter_fasta(min_len = 200, max_len = 580):
    '''Remove sequences that are too long or too short, also sequenes that have X in them'''
    infile = join(INTERMEDIATE, 'brenda_2019_2', '1_1_3__BRENDA_sequences_2019_2.fasta')
    outfile = join(INTERMEDIATE, 'brenda_2019_2', '1_1_3__BRENDA_sequences_filtered_2019_2.fasta')

    retained = 0
    removed_len = 0
    removed_x = 0
    removed_m = 0
    with open(outfile, 'w') as f:
        for record in SeqIO.parse(infile, 'fasta'):
            header = record.description
            seq = str(record.seq).lower()

            if min_len <= len(seq) <= max_len and 'x' not in seq and seq[0] == 'm':
                retained += 1
                f.write('>%s\n%s\n' % (header, seq))
            elif 'x' in seq:
                removed_x += 1
            elif seq[0] != 'm':
                removed_m += 1
            else:
                removed_len += 1


# create a fasta file with the 1.1.3.n sequences
make_fasta()

# filter out sequences that are too short or too long
filter_fasta()


infile = join(INTERMEDIATE, 'brenda_2019_2', '1_1_3__BRENDA_sequences_2019_2.fasta')
outfile1 = join(FINAL, 'brenda_2019_2', 'ec_uid_org_from_fasta_2019_2.tsv')
outfile2 = join(FINAL, 'brenda_2019_2', '1-1-3-n_identifier_info_2019_2.tsv')

if (not exists(outfile1)) or (not exists(outfile2)):
    data = get_uid_from_fasta(filepath=infile)
    write_outfile(data, filepath=outfile1)
    write_info_outfile(data, filepath=outfile2)

else:
    print('Outfiles exist, skipping.')

Outfiles exist, skipping.


### Filter out only the 1.1.3.15 sequences from both database versions and get their UniRep representations 

In [5]:
def filter_for_ec(infile, outfile, ec):
    '''
    Take a fasta file as input and retain only sequences
    belonging to a certain ec class. Also remove duplicates.
    '''
    # parse the fasta file and only keep sequenes from specified EC
    # the dictionary data structrue effectively removes (header) duplicates
    out_data = {}
    for record in SeqIO.parse(infile, 'fasta'):
        header = record.description
        seq = record.seq
        
        if header.split(';')[2] == ec:
            out_data[header] = seq
       
    # generate the output lines
    outlines = []
    for k, v in out_data.items():
        outlines.append('>{}\n{}'.format(k, v))
    
    # write to disk
    with open(outfile, 'w') as f:
        f.write('\n'.join(outlines))


def compute_unirep_representations(fasta_file, rep_file):
    '''
    Compute sequence representations from fasta file.
    '''
    tf.keras.backend.clear_session()
    
    inf_obj = BatchInference(batch_size=500)
    df = inf_obj.run_inference(filepath=fasta_file)
    df.to_csv(rep_file, sep='\t')
    
    return df.values

##########
## 2017 ##
##########
raw_fasta = join(INTERMEDIATE, 'brenda_2017_1', '1_1_3__BRENDA_sequences_2017_1.fasta')
filtered_fasta = join(INTERMEDIATE, 'brenda_2017_1', '1_1_3_15_BRENDA_sequences_2017_1.fasta')
rep_file = join(INTERMEDIATE, 'brenda_2017_1', '1_1_3_15_BRENDA_sequences_2017_1_unirep.csv')

# filter fasta file to only retain 1.1.3.15
if not exists(filtered_fasta):
    filter_for_ec(raw_fasta, filtered_fasta, ec='1.1.3.15')

# compute the unreip representations
if not exists(rep_file):
    compute_unirep_representations(filtered_fasta, rep_file)


##########
## 2019 ##
##########
raw_fasta = join(INTERMEDIATE, 'brenda_2019_2', '1_1_3__BRENDA_sequences_2019_2.fasta')
filtered_fasta = join(INTERMEDIATE, 'brenda_2019_2', '1_1_3_15_BRENDA_sequences_2019_2.fasta')
rep_file = join(INTERMEDIATE, 'brenda_2019_2', '1_1_3_15_BRENDA_sequences_2019_2_unirep.csv')

# filter fasta file to only retain 1.1.3.15
if not exists(filtered_fasta):
    filter_for_ec(raw_fasta, filtered_fasta, ec='1.1.3.15')

# compute the unreip representations
if not exists(rep_file):
    compute_unirep_representations(filtered_fasta, rep_file)

FileNotFoundError: [Errno 2] No such file or directory: '/data/projects/test/analyze_1.1.3.15/data/intermediate/brenda_2017_1/1_1_3__BRENDA_sequences_2017_1.fasta'

### Compute EC 1.1.3.15 identity matrix using pairwise alignments

In [ ]:

# Get a list of all selected
def get_sequences():
    '''
    Get all sequencs as a dictionary
    '''
    seq_data = {}
    infile = join(INTERMEDIATE, 'brenda_2017_1', '1_1_3_15_BRENDA_sequences_2017_1.fasta')
    for record in SeqIO.parse(infile, 'fasta'):
        header = record.description
        seq = record.seq
        seq_data[header] = str(seq)

    return seq_data


def pairwise_align(id1, id2, seq1, seq2):
    '''
    Perform pairwise alignment.
    '''
    records = '>%s\n%s\n>%s\n%s' % (id1, seq1, id2, seq2) #prepare 'virtual' FASTA file

    records_handle = io.StringIO(records) #turn string into a handle
    tempdata = records_handle.getvalue()
    muscle_cline = MuscleCommandline()
    stdout, stderr = muscle_cline(stdin=tempdata)
    aln = AlignIO.read(io.StringIO(stdout), "fasta")

    output = []
    for entry in aln:
        output.append(entry.id.lstrip('>'))
        output.append(entry.seq)
    return output


def pair_ident(Seq1, Seq2, single_gaps=True):
    '''
    Takes two aligned sequences and returns their percent identity.
    Assumes that Seq1 and Seq2 are sequence strings
    '''

    l=0.0 # counts alignment length, excluding identical gaps, but including single gaps
    n=0.0 # count number of single gaps
    i=0.0 # counts identity hits


    for j in range(len(Seq2)):
        if Seq1[j] == '-' and Seq2[j] == '-': #DON'T count identical gaps towards alignment length
            pass
        else:
            if Seq2[j] == Seq1[j]:
                i += 1 #count matches
            elif Seq2[j] == '-' or Seq1[j] == '-':
                n += 1 #count number of single gaps
            l += 1 #count total length with single gaps

    if single_gaps is True: #include single gaps
        percent = round(100*(i/l),1) #calculate identity

    elif single_gaps is False: #exclude single gaps
        if n >= l:
            percent = 0.0
        else:
            percent = round(100*(i/(l-n)),1) #calculate identity

    return percent


def worker(package):
    '''
    '''
    id1, seq1, id2, seq2 = package

    # sometimes there is no closest match (i.e. no characterized sequence)
    if id2 == 'None':
        return (id1, id2, 0)
    
    # sequences that are too long make muscle fail, so ignore those
    elif len(seq1) > 10000 or len(seq2) > 10000:
        return (id1, id2, 0)


    output = pairwise_align(id1='>%s'%id1, id2='>%s'%id2, seq1=seq1, seq2=seq2)

    aln_seq1 = output[1]
    aln_seq2 = output[3]

    identity = pair_ident(Seq1=aln_seq1, Seq2=aln_seq2, single_gaps=True)
    
    return (id1, id2, identity)



def get_packages():
    '''
    Generator for assembling sequence packages.
    '''
    # get all sequences
    seq_data = get_sequences()
    
    # collect all the data, for later disribution to workeres
    for i, id1 in enumerate(seq_data.keys()):
        for j, id2 in enumerate(seq_data.keys()):
            if i < j:
                continue

            seq1 = seq_data[id1]
            seq2 = seq_data[id2]

            # keep only the uniprot identifiers
            uid1 = id1.split(';')[0]
            uid2 = id2.split(';')[0]

            # store the sequences and identifiers
            yield ((uid1, seq1, uid2, seq2))

            
def compute_matrix():
    '''
    Do pairwise alignments and compute identities.
    '''
    # get all sequences
    seq_data = get_sequences()

    # generate all pairs and align them
    total_alns = (len(seq_data.keys())**2-len(seq_data.keys())) / 2 + len(seq_data.keys())
    print('Total {} alignments to do'.format(total_alns))

    # carry out the alignments
    num_cores = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=num_cores) as pool:
        results = list(tqdm(pool.imap(worker, get_packages()), total=total_alns))

    # put the resulst into identiy matrix
    identity_data = {}
    for res in results:
        uid1, uid2, identity = res

        if identity_data.get(uid1) is None:
            identity_data[uid1] = {}

        if identity_data.get(uid2) is None:
            identity_data[uid2] = {}

        identity_data[uid1][uid2] = identity
        identity_data[uid2][uid1] = identity
            
    return identity_data


def save_identity_matrix_all(data, filepath):
    '''
    Save the identity data as a flatfile matrix.
    '''
    # Convert identity values to data frame and save to flatfile
    df = pd.DataFrame(data)
    df = df.fillna(0)
    df.to_csv(path_or_buf=filepath, sep='\t')



outfile = join(FINAL, 'brenda_2017_1', 'all_identity_matrix_ident.tsv')
if not exists(outfile):
    identity_data = compute_matrix()
    save_identity_matrix_all(identity_data, filepath=outfile)
else:
    print('Outfile exists, skipping.')

### Download PFAM domain info for sequences from BRENDA version 2017.1

In [ ]:
def dlfile(folder, filename, url):
    '''
    Download a web page if the corresponding file does not exist.
    '''

    # Open the url
    try:
        out_path = join(folder, filename)
        if not exists(out_path):
            f = urlopen(url)
            print("downloading " + url)

            # Open local file for writing
            with open(out_path, "wb") as local_file:
                local_file.write(f.read())
            time.sleep(1)

    #handle errors
    except HTTPError as e:
        print("HTTP Error:", e.code, url)
    
    except URLError as e:
        print("URL Error:", e.reason, url)
        

def get_pfam_from_file(filepath):
    '''
    Parse out pfam data from a file
    '''
    # open the uniprot and uniparc pages and append them
    with open(filepath, 'r') as f:
        document = f.read()

        
    # search through the combined document for identifiers
    m = re.findall('(PF[0-9]{5}|CL[0-9]{4})', document)

    # loop through the search result and keep unique identifiers
    pfam_ids = set([])
    for pid in m:
        if pid == '':
            continue
                
        pfam_ids.add(pid)
        
    return pfam_ids
        
    
    
def get_pfam_for_uids(uids, filepath):
    '''
    Query the UniProt database to get protein domains for 
    a list of protein identifiers.
    '''

    data = {'uid':[], 'pfam':[]}

    for uid in uids:

        # download uniprot file
        uniprot_url = 'https://www.uniprot.org/uniprot/'
        dlfile(folder=filepath, filename='%s_uniprot.html' % uid, url=uniprot_url+uid)

        # query uniprot file
        pfam_ids = get_pfam_from_file(join(filepath, '%s_uniprot.html' % uid))


        # download uniparc overview file
        uniparc_url = 'https://www.uniprot.org/uniparc/?query=%s' % uid
        overview_filename = join(filepath, '%s_uniparc_overview.html' % uid)
        dlfile(folder=filepath, filename='%s_uniparc_overview.html' % uid, url=uniparc_url)

        # find the reference for the alternate identifier and download that file
        with open(overview_filename, 'r') as f:
            document = f.read()

            m = re.search('class="entryID"><a href="/uniparc/([a-zA-Z0-9]+)">', document)
            new_target_url = 'https://www.uniprot.org/uniparc/%s' % m.group(1)
            dlfile(folder=filepath, filename='%s_uniparc.html' % uid, url=new_target_url)

        # query uniparc file
        pfam_ids2 = get_pfam_from_file(join(filepath, '%s_uniparc.html' % uid))


        data['uid'].append(uid)
        data['pfam'].append(','.join(sorted(list(pfam_ids.union(pfam_ids2)))))

    
    return pd.DataFrame(data)

In [ ]:
outfile = join(FINAL, 'brenda_2017_1', 'pfam_info_2017_1.tsv')
if not exists(outfile):
    # load a list of the identifiers I want
    filepath = join(FINAL, 'brenda_2017_1', 'ec_uid_org_from_fasta_2017_1.tsv')
    uid_ec = pd.read_csv(filepath, sep='\t').drop_duplicates()

    # only keep 1.1.3.15
    data_subset = uid_ec[uid_ec['ec']=='1.1.3.15']
    uids = data_subset.uid.values
    
    display(data_subset.head())
    display(len(uids))

    # download uniparc and uniprot for each
    filepath = join(RAW_EXTERNAL, 'pfam')
    if not exists(filepath):
        os.mkdir(filepath)
    
    data_frame = get_pfam_for_uids(uids, filepath)
    data_frame.to_csv(outfile, sep='\t', index=False)
    
else:
    print('Outfile exists, skipping.')

### Download PFAM domain info for sequences from BRENDA version 2019.2

In [ ]:
outfile = join(FINAL, 'brenda_2019_2', 'pfam_info_2019_2.tsv')
if not exists(outfile):
    # load a list of the identifiers I want
    filepath = join(FINAL, 'brenda_2019_2', 'ec_uid_org_from_fasta_2019_2.tsv')
    uid_ec = pd.read_csv(filepath, sep='\t').drop_duplicates()

    # only keep 1.1.3.15
    data_subset = uid_ec[uid_ec['ec']=='1.1.3.15']
    uids = data_subset.uid.values
    
    display(data_subset.head())
    display(len(uids))

    # download uniparc and uniprot for each
    filepath = join(RAW_EXTERNAL, 'pfam')
    if not exists(filepath):
        os.mkdir(filepath)
    
    data_frame = get_pfam_for_uids(uids, filepath)
    data_frame.to_csv(outfile, sep='\t', index=False)
    
else:
    print('Outfile exists, skipping.')